In [1]:
import os

class Country():
    """Country class which specifies the (top 10) ranking of a country.
    Optionally gives the score this country achieved in the semifinal."""
    def __init__(self, country, ranking_list, semifinal_score=0):
        self.country = country
        self.ranking_list = ranking_list
        self.semifinal_score = semifinal_score

    def get_ranking_list(self):
        return self.ranking_list

    def get_semifinal_score(self):
        return self.semifinal_score

    def get_ith_ranking(self, i):
        return self.ranking_list[i-1]

class Rule():
    """Rule class consisting of a scoring vector and a vector length.
    Scoring vector is specified from largest to smallest score.
    Will be expanded once the thesis gets to more advanced
    material."""
    def __init__(self, scoring_vector, vector_length):
        self.scoring_vector = scoring_vector
        self.vector_length = vector_length

    def get_scoring_vector(self):
        return self.scoring_vector

    def get_vector_length(self):
        return self.vector_length

class Contest():
    """Contest class includes everything to compute the original outcome
    of a certain year's ESC. Later will also be able to solve the problem(s)
    specified in the thesis proposal."""
    def __init__(self, country_list, voting_rule, nr_of_semifinals=0):
        self.country_list = country_list
        self.voting_rule = voting_rule
        self.nr_of_countries = len(country_list)
        self.nr_of_semifinals = nr_of_semifinals
        
    def compute_result(self):
        """Computes the result and outputs a dict with each country that scored
        more than 0 points."""
        result = dict()
        rule_vector = self.voting_rule.get_scoring_vector()
        rule_length = self.voting_rule.get_vector_length()
        for country in self.country_list:
            ranking_list = country.get_ranking_list()
            rank = 0
            for c in ranking_list:
                if rank + 1 > rule_length:
                    break
                if c not in result:
                    result[c] = 0
                result[c] += rule_vector[rank]
                rank += 1

        return result

    def print_result(self):
        """Prints the result in a ranking list format"""
        result = self.compute_result()
        for element in sorted(result, key=result.get, reverse=True):
            print(element, result[element])


In [2]:
import csv

def import_scores(file_name = 'votes.csv'):
    """ This function is hardcoded. Reads and puts the votes for the finals
    from 1975 to 2015 in a dictionary. This dict has 2 levels:
    Top level: key = year, value = dict of scoring countries
    Bottom level: key = scoring country, value = ranked list of preferences (top 10)"""
    f = open(file_name, "r")
    ff = csv.reader(f)

    ESC = dict()
    for year in range(1975,2016):
        ESC[str(year)] = dict()

    for row in ff:
        if row[0] == 'year':
            continue
        if int(row[0]) not in range(1975,2016) or row[1] != 'final' or row[6] == '0':
            continue
        if row[2] not in ESC[row[0]]:
            ESC[row[0]][row[2]] = dict()
        ESC[row[0]][row[2]][row[3]] = int(row[6])
        
    scores_to_rankings(ESC)
    return ESC

def scores_to_rankings(ESC_dict):
    """ ESC dict contains a year dict, which contains country dicts with a dict
    using scored_country/score tuples. This function turns that into a list
    ranking the countries instead. """
    for year in ESC_dict:
        for country in ESC_dict[year]:
            score_dict = ESC_dict[year][country]
            ranking_list = []
            for scored_country in sorted(score_dict, key=score_dict.get, reverse=True):
                ranking_list.append(scored_country)
            ESC_dict[year][country] = ranking_list
    pass

def dict_to_classes(d, voting_rule):
    """Takes as input a dict outputted by import_scores(),
    and returns a dict with objects of class Contest."""
    ESC_classified = dict()
    for year in range(1975,2016):
        ESC_classified[str(year)] = None
        
    for year in d:
        country_list = []
        for country in d[year]:
            new_country = Country(country, d[year][country])
            country_list.append(new_country)
        ESC_classified[year] = Contest(country_list, voting_rule)
    
    return ESC_classified

In [4]:
rule_1975_2015 = Rule([12,10,8,7,6,5,4,3,2,1], 10)
ESC_classified = dict_to_classes(import_scores(), rule_1975_2015)
ESC_1975 = ESC_classified['1975']
ESC_1975.print_result()

nl 152
gb 138
it 115
fr 91
lu 84
ch 77
fi 74
se 72
ie 68
es 53
il 40
mt 32
yu 22
mc 22
be 17
pt 16
de 15
no 11
tr 3
